## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries.

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import sys
sys.path.append('../USGStools')
from StreamStats_API_Scraper import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

In [2]:
state='NY' #The state abbreviation in uppercase

path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\StreamStats\results\SalmonCreek' #Specify the location of the shapefile containing the lat/lon of points on the stream grid
name='Confluences.shp' #The name of the shapefile

##### Load the shapefile:

In [3]:
use_epsg='4326' #Specify a consistent coordinate reference system

gdf=gpd.read_file(os.path.join(path, name)) #Read the shapefile as a geopandas dataframe

gdf=gdf.to_crs({'init': 'epsg:{0}'.format(use_epsg)})

geom=gdf.geometry #Extract the shapley geometry for the outlets in the shapefile

print(geom.head(2))

0    POINT (-74.54704371705137 44.99176157571224)
1      POINT (-74.5534966416888 44.9939474520524)
Name: geometry, dtype: object


### Run the API tool for each point:

In [ ]:
pp_dic = {} #Dictionary to store the outlet flow frequency data dictionaries

watershed_poly_dic= {} #Dictionary to store the catchment polygons (catchment boundaries)

pp_fail=[] #List to store outlet locations whose flow frequency/catchment polygons were not calculated

pp_dic, watershed_poly_dic=snappoint_analysis(geom, state, status=True) #Run the snappoint function for all catchment outlets within the shapefile and for the specified state. Option: set status=False to hide print statements

-74.54704371705137 44.99176157571224
Fetched Peak Flows
-74.5534966416888 44.993947452052396
Fetched Peak Flows
-74.54824694774688 44.98393464881284
Fetched Peak Flows
-74.57079689624692 44.98771242150379
Fetched Peak Flows
-74.54016232617454 44.979761483211284
Fetched Peak Flows
-74.55104795706318 44.98259540136499
Fetched Peak Flows
-74.54043540176825 44.97733205311249
Fetched Peak Flows
-74.56655189428909 44.978694435906455
Fetched Peak Flows
-74.54569919055172 44.96925134187206
Fetched Peak Flows
-74.54634103170272 44.968263675961765
Fetched Peak Flows
-74.55502432363302 44.96046611402977
Fetched Peak Flows
-74.54114777275831 44.96752277022507
Line 28: Expecting value: line 1 column 1 (char 0
while loop: watershed_data count: 1
Fetched Peak Flows
-74.54522681636762 44.96483849234625
Fetched Peak Flows
-74.56062566765343 44.95751714421745
Fetched Peak Flows
-74.53885665990144 44.968593779156855
Line 28: Expecting value: line 1 column 1 (char 0
while loop: watershed_data count: 1
Fet

### Construct a summary table of the flow frequency data for each outlet:

In [5]:
ffdata=ff_summary(pp_dic) #Run this function to construct the summary table for all outlet locations

ffdata.head()    

,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
RI,,,,,,,,,,,,,,,,,,,,,
1.25,314.504420,16.851665,302.174411,6.794247,14.467800,22.021159,289.006541,8.746245,19.987076,249.688014,...,1.723058,2.332864,2.105253,2.270258,2.764553,4.314330,4.072714,6.485691,3.007013,1.730021
1.50,380.103839,20.755793,365.539756,8.393763,18.020295,27.374481,349.355648,10.873940,24.850808,301.929229,...,2.117841,2.862039,2.611856,2.793417,3.417114,5.368322,5.007398,7.926910,3.712904,2.145564
2.00,465.022384,26.069840,447.651073,10.594934,22.933862,34.757811,427.546803,13.843697,31.546468,369.738347,...,2.665305,3.590001,3.328109,3.515409,4.323042,6.844697,6.292693,9.872867,4.696587,2.733252
5.00,696.659383,41.009705,671.767545,16.820338,36.978923,55.659253,640.793451,22.214070,50.642560,554.628450,...,4.189206,5.612619,5.343700,5.538878,6.883312,11.056032,9.862521,15.264058,7.443271,4.386921
10.00,860.643658,52.169503,830.448611,21.516015,47.776254,71.442667,791.625454,28.481823,65.282378,685.274142,...,5.300664,7.079838,6.853089,7.033297,8.808672,14.287494,12.450090,19.146062,9.461647,5.624141


###  Save the results:

##### As a CSV:

In [6]:
ffdata.to_csv(os.path.join(path,'flow_frequency_data.csv')) #Save the results as a csv

##### As a geojson:

In [7]:
for i in pp_dic.keys():
    watershed_poly_dic[i]['features'][0]['ffcurve']=pp_dic[i]
    
with open('results\SalmonCreek\catchment_poly.geojson', 'w') as f:
   dump(watershed_poly_dic, f)        

# END